In [ ]:
import csv, requests, re
from urllib.request import urlretrieve
from random import randint
import glob
from time import sleep
from bs4 import BeautifulSoup as bs

In [ ]:
def rand():
    return randint(0,5)*0.1

In [ ]:
def get_links(searchterm):
    ### get all application numbers according to the search term.

    searchterm = searchterm.replace(" ","+")
    url = "https://register.epo.org/smartSearch?searchMode=smart&query="+searchterm
    
    session = requests.Session()
    response = session.get(url)
    cookie = session.cookies.get_dict()
        
    r = requests.get(url)
    soup = bs(r.text)
    
    pgs = re.search("[0-9]+", re.escape(soup.find("a", {"class" : "paginationLast"}).get("href"))).group(0)

    appnumbers = []

    for nr in range(1,int(pgs)):
        print(nr, end=" ")

        inputweb = "https://register.epo.org/smartSearch?page="+str(nr)+"&query="+searchterm+"&searchMode=smart"
        # print inputweb
        
        sleep(rand())
        
        header = {'User-Agent': 'Mozilla/5.0 (X11; Fedora; Linux x86_64; rv:36.0) Gecko/20100101 Firefox/36.0'}
        r = requests.get(inputweb, headers=header, cookies=cookie)
        soup = bs(r.text)

        link = soup.find("table", {"class": "application"}).find_all("a")
        
        for l in link:
            l = l.get("href")
            if 'application?number' in l:
                appNUM = l.replace("application?number=","")
                appnumbers.append(appNUM)

    return appnumbers

## Insert your search terms & collect the application numbers

[test your search term first](https://register.epo.org/regviewer). The search is being performed in the WHOLE application. The searchterm has to consist of min 3 characters<br>
see the [usage of wildcards](https://register.epo.org/help?topic=truncation&lng=en)

In [ ]:
searchterms = ['"Panama city"','"The Valley"',"Apia","Victoria Mahe","Tortola","Nassau"]

In [ ]:
docnrs = []

for s in searchterms:
    print(s)
    docnrs.extend(get_links(s))

### scrape the XMLs

In [ ]:
directory = "./"

for appnum in docnrs:
    sleep(rand())
    urlretrieve("https://register.epo.org/download?number="+appnum+"&tab=main&xml=st36", directory+appnum+".xml")